In [1]:
print('Hello LangChain')

Hello LangChain


In [2]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#print(OPENAI_API_KEY)

In [3]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("human", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="LangServe는 무엇인가요? 자세하게 설명해주세요")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [4]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000002C073BDE240> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002C073C3E270> root_client=<openai.OpenAI object at 0x000002C073699B50> root_async_client=<openai.AsyncOpenAI object at 0x000002C07372E780> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [5]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print(response)
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
content='LangServe는 개발자가 대규모 언어 모델(LLM, Large Language Model)을 쉽게 배포하고 관리할 수 있도록 지원하는 오픈소스 라이브러리입니다. LangServe를 사용하면 개발자는 언어 모델을 API 형태로 제공하여 다양한 애플리케이션에서 쉽게 사용할 수 있습니다.\n\nLangChain 기반으로 구축된 LangServe는 모델의 추론 기능을 HTTP API 서버로 제공하며, 여러 엔드포인트를 제공하여 모델의 기능을 활용할 수 있습니다. 이를 통해 LangServe는 다음과 같은 이점을 제공합니다.\n\n- **쉬운 배포**: LangServe를 사용하면 언어 모델을 쉽고 빠르게 배포할 수 있습니다. 모델을 API 서버로 변환하여 여러 환경에서 손쉽게 활용할 수 있습니다.\n\n- **유연성**: LangServe는 다양한 언어 모델과 호환되며, 여러 API 엔드포인트를 제공하여 모델의 다양한 기능을 활용할 수 있습니다.\n\n- **관리 편의성**: LangServe는 모델의 성능을 모니터링하고, 업데이트를 쉽게 적용할 수 있는 관리 기능을 제공합니다.\n\nLangServe의 주요 기능은 다음과 같습니다.\n\n1. **모델 배포**: LangServe를 사용하면 선택한 언어 모델을 API 서버로 쉽게 배포할 수 있습니다.\n\n2. **API 엔드포인트 제공**: LangServe는 여러 API 엔드포인트를 제공하여 모델의 다양한 기능을 활용할 수 있습니다.\n\n3. **입력/출력 관리**: LangServe는 모델에 입력 데이터를 제공하고, 모델의 출력을 받아서 처리하는 기능을 제공합니다.\n\n4. **성능 모니터링**: LangServe는 모델의 성능을 실시간으로 모니터링할 수 있는 기능을 제공합니다.\n\n5. **업데이트 지원**: LangServe는 모델의 업데이트를 쉽게 적용할 수 있도록 지원합니다.\n\nLangServ

### LCEL
- Prompt + LLM을 Chain으로 연결하기

In [6]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    You are an expert in AI Expert. Answer the question.
    <Question>: {input}에 대해 쉽게 설명해주세요.
    """)
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n    You are an expert in AI Expert. Answer the question.\n    <Question>: {input}에 대해 쉽게 설명해주세요.\n    ')

In [7]:
# chain 연결 (LCEL)
chain = prompt | llm
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [8]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL) prompt + Lim + outputParser
output_parser = StrOutputParser()

chain2 = prompt | llm | output_parser
print(type(chain2))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [9]:
# chain 호출
try:
    result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
content="인공지능 모델의 학습 원리는 사람의 뇌가 학습하는 방식과 유사합니다. 컴퓨터가 데이터를 통해 배우고, 패턴을 발견하며, 미래의 새로운 데이터에 대해 예측하거나 결정을 내릴 수 있도록 하는 것입니다.\n\n쉽게 설명하면 다음과 같습니다:\n\n1. **데이터 수집**: 인공지능 모델을 훈련시키기 위해 많은 데이터를 수집합니다. 이 데이터는 과거의 경험이나 관찰을 통해 얻어진 정보입니다.\n\n2. **데이터 분석**: 수집된 데이터를 분석하여 패턴이나 규칙을 찾습니다. 예를 들어, 고양이와 강아지의 사진을 보여주며 이들이 각각 어떤 동물인지를 컴퓨터가 구분할 수 있도록 하는 것입니다.\n\n3. **모델 훈련**: 컴퓨터는 알고리즘을 통해 이 데이터를 분석하고, 스스로 규칙이나 패턴을 학습합니다. 이 과정에서는 '오답을 맞았을 때 스스로 수정하는 과정'이 포함됩니다. 즉, 처음에는 정확하지 않을 수 있지만, 반복된 훈련을 통해 정확도가 높아집니다.\n\n4. **성능 평가**: 학습된 모델이 얼마나 정확한지 평가합니다. 이는 마치 시험을 쳐서 점수를 매기는 것과 같습니다.\n\n5. **예측 또는 결정**: 학습된 모델을 사용하여 새로운 데이터에 대해 예측하거나 결정을 내립니다. 예를 들어, 새로운 동물의 사진을 보여주었을 때, 고양이인지 강아지인지를 구분하는 것입니다.\n\n이를 비유로 설명하면, 인공지능 모델의 학습 원리는 어린 아이가 걷는 법을 배우는 것과 유사합니다. 처음에는 잘 걷지 못하지만, 계속해서 시도하고, 실패하고, 배운 것을 통해 조금씩 잘 걷게 됩니다. 인공지능 모델도 이와 같은 원리로 학습하고, 주어진 문제에 대해 더 나은 해결책을 찾아가게 됩니다." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 356, 'prompt_tok

In [10]:
# chain 호출
try:
    result = chain2.invoke({"input": ": LangChain의 Products(제품)는 어떤 것들이 있나요? 예를 들어 LangSmith, LangServe 같은 Product가 있어"})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
LangChain은 다양한 제품과 도구를 제공하여 개발자와 기업이 인공지능(AI) 기술을 보다 쉽게 통합하고 활용할 수 있도록 지원합니다. LangChain의 주요 제품 중 LangSmith와 LangServe에 대해 설명드리겠습니다.

### LangSmith

LangSmith는 LangChain에서 제공하는 플랫폼으로, 개발자가 랭귀지 모델을 더 쉽게 개발, 테스트, 배포할 수 있도록 설계되었습니다. LangSmith를 통해 개발자는 다음과 같은 작업을 보다 효율적으로 수행할 수 있습니다:

- **모델 개발**: LangSmith는 개발자가 자연어 처리(NLP) 모델을 만들고 미세 조정할 수 있는 환경을 제공합니다.
- **워크플로우 관리**: 개발자는 LangSmith를 사용하여 복잡한 워크플로를 관리할 수 있습니다. 여기에는 여러 단계의 처리, 조건부 로직, 외부 서비스와의 통합 등이 포함될 수 있습니다.
- **디버깅 및 테스팅**: LangSmith는 모델의 성능을 테스트하고 디버깅하는 데 도움이 되는 도구들을 제공합니다.

즉, LangSmith는 개발자가 랭귀지 모델을 개발하고 운영하는 전 과정을 지원하는 통합 플랫폼입니다.

### LangServe

LangServe는 LangChain에서 제공하는 또 다른 제품으로, 랭귀지 모델의 배포 및 운영을 간소화하는데 중점을 둡니다. LangServe를 통해 개발자와 기업은 다음과 같은 혜택을 얻을 수 있습니다:

- **모델 배포**: LangServe는 랭귀지 모델을 쉽고 빠르게 배포할 수 있는 방법을 제공합니다. 이를 통해 모델을 실제 환경에서 운영할 때 필요한 인프라와 복잡성을 줄일 수 있습니다.
- **API 기반 접근**: 배포된 모델은 API를 통해 접근할 수 있어, 다른 애플리케이션과의 통합이 용이합니다.
- **모니터링 및 관리**: LangServe는 배포된 모델의 성능을 모니터링하고 관리할 수 있는 기능도 제공합니다.

LangServe는 본질적으로 모

### Runnable의 stream() 함수 호출

In [11]:
# 스트리밍 출력을 위한 요청
try:
    answer = chain2.stream({"input": "인공지능 모델의 학습 원리를 자세하게 설명해 주세요."})

    # 스트리밍 출력
    # print(answer)
    for token in answer:
        # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
        print(token, end="", flush=True)
except Exception as e:
    print(f"오류 발생: {e}")

인공지능 모델의 학습 원리를 쉽게 설명해 드리겠습니다.

인공지능 모델의 학습 원리는 사람의 뇌가 학습하는 원리와 유사합니다. 사람은 경험을 통해 학습하고, 컴퓨터도 데이터를 통해 학습합니다.

1. **데이터 수집**: 인공지능 모델을 학습시키기 위해서는 많은 데이터가 필요합니다. 이 데이터는 문제에 대한 답이 포함된 형태여야 합니다. 예를 들어, 고양이와 강아지의 사진을 분류하는 모델을 만든다면, 고양이와 강아지의 사진 데이터가 필요합니다.

2. **데이터 전처리**: 수집한 데이터를 모델이 이해할 수 있도록 가공하는 과정입니다. 이 과정에서는 데이터의 품질 개선, 변환, 그리고 필요한 속성 추출 등이 포함됩니다.

3. **모델 선택**: 문제의 성격에 따라 적합한 모델을 선택합니다. 예를 들어, 이미지 분류 문제에는 합성곱 신경망(CNN), 자연어 처리 문제에는 순환 신경망(RNN)이나 트랜스포머 등이 적합합니다.

4. **모델 학습**: 선택한 모델에 데이터를 입력하여 모델의 파라미터를 조정하는 과정입니다. 이 과정에서는 모델이 데이터로부터 패턴이나 규칙을 스스로 찾아내도록 합니다. 학습 과정은 보통 다음과 같은 단계로 진행됩니다:
   - **예측**: 모델이 입력된 데이터에 대해 예측을 합니다.
   - **오차 계산**: 예측 결과와 실제 값 사이의 오차를 계산합니다. 이 오차를 손실 함수(loss function)로 표현합니다.
   - **역전파**: 오차를 줄이기 위해 모델의 파라미터를 어떻게 조정해야 하는지 계산하는 과정입니다. 이 과정은 역전파(backpropagation)를 통해 이루어집니다.
   - **파라미터 업데이트**: 계산된 오차를 줄이는 방향으로 모델의 파라미터를 업데이트합니다. 이 과정은 최적화 알고리즘(예: SGD, Adam 등)을 통해 이루어집니다.

5. **모델 평가**: 학습된 모델의 성능을 평가합니다. 이는 테스트 데이터를 사용하여 모델의 예측 성능을 측정하는 과정입니다. 평가 지표에는 정확도, 정밀도, 재

### Multi Chain
* 첫번째 Chain의 출력이, 두번째 Chain의 입력이 된다.
* 두개의 Chain과 Prompt + OutputParser LCEL로 연결하기

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가  입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 3문장으로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

In [13]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 입력 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행: "SF" 장르의 영화 추천 및 줄거리 요약
    response = chain2.invoke({"genre": "Drama"})
    print(response)  
except Exception as e:
    print(f"오류 발생: {e}")

'루이스 칸: 모든 건축은 건축가부터'입니다.

루이스 칸은 20세기 현대 건축의 거장 중 한 명으로, 그의 건축물은 기능성과 미학을 조화롭게 결합한 것으로 유명합니다. 영화는 칸의 개인적인 삶과 직업적인 삶, 그리고 그가 남긴 건축 유산을 탐구하며, 그의 창의성과 열정, 그리고 그가 직면한 어려움을 조명합니다. 칸의 삶과 작품을 통해 인간의 삶과 창의성에 대한 깊은 통찰을 제공합니다.


### PromptTemplate 여러개 연결하기기

In [14]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})
print(response)

ChatGPT 모델의 학습 원리는 다음과 같습니다.

ChatGPT는 대규모 언어 데이터셋을 기반으로 하는 딥러닝 모델로, 주어진 문맥에 따라 다음에 올 가능성이 높은 단어를 예측하도록 학습합니다. 이를 위해 모델은 자기 자신의 출력값을 재귀적으로 입력값으로 사용하며, 이를 통해 긴 문장이나 대화에서도 자연스러운 응답을 생성할 수 있습니다. 이러한 학습 방식은 모델이 다양한 언어 패턴과 구조를 학습하고, 상황에 맞는 적절한 응답을 생성할 수 있도록 도와줍니다.

ChatGPT 모델의 장점은 다음과 같습니다.

*   자연스러운 대화 생성: ChatGPT는 긴 대화에서도 자연스럽고 일관된 응답을 생성할 수 있습니다.
*   다양한 주제 이해: ChatGPT는 다양한 주제와 분야에 대해 이해하고 응답할 수 있습니다.
*   높은 적응성: ChatGPT는 상황에 맞게 적응하여 응답할 수 있습니다.

ChatGPT 모델과 비슷한 AI 모델은 다음과 있습니다.

*   LLaMA
*   PaLM
*   BERT
*   RoBERTa


In [15]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
    {"model_name": "llama-4", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

['GPT-4 모델의 학습 원리를 2 문장으로 요약해서 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 3 문장으로 요약해서 한국어로 답변해 주세요.', 'llama-4 모델의 학습 원리를 4 문장으로 요약해서 한국어로 답변해 주세요.']


In [16]:
# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

for prompt in formatted_prompts:
    response = llm.invoke(prompt) #AIMessage
    print(response.content)

GPT-4 모델은 대규모의 텍스트 데이터를 학습하여 언어 패턴과 관계를 파악하고, 이를 바탕으로 주어진 입력에 대해 적절한 응답을 생성하도록 설계되었습니다. GPT-4는 이전 모델인 GPT-3와 마찬가지로 트랜스포머 아키텍처를 기반으로 하며, 강화된 학습 데이터와 최적화된 학습 알고리즘을 통해 성능이 개선되었습니다.
Gemma는 컴퓨터가 자연어로 텍스트를 생성하고 이해할 수 있도록 설계된 언어 모델입니다. 대규모 텍스트 데이터 세트에 대해 학습되었으며, 다양한 자연어 처리 작업을 수행할 수 있습니다. 예를 들어, 텍스트 생성, 요약, 번역, 대화형 AI 등의 작업에 활용될 수 있습니다.
llama-4 모델은 메타에서 개발한 대규모 언어 모델입니다. 이 모델은 수십억 개의 매개변수를 가지고 있으며, 방대한 양의 텍스트 데이터를 학습하여 자연어 처리 능력을 습득합니다. 학습 과정에서 모델은 입력된 텍스트의 패턴과 구조를 파악하고, 이를 기반으로 다음에 나타날 단어나 구절을 예측하도록 훈련됩니다. 이를 통해 llama-4 모델은 다양한 자연어 처리 작업에서 높은 성능을 발휘할 수 있습니다.


### ChatPromptTemplate
* SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate 사용

In [17]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 {topic} 전문가입니다. 명확하고 자세하게 설명해 주세요."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="딥러닝은 무엇인가요?")

# LLM 호출
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
response = llm.invoke(messages)

# 결과 출력
print(response.content)

 

## Definition 

딥러닝은 인공신경망을 사용하여 데이터를 분석하고 패턴을 찾는 머신러닝의 한 분야입니다. 딥러닝은 데이터의 복잡한 패턴을 학습하고 높은 수준의 추상화를 달성할 수 있는 심층 신경망을 사용합니다. 이러한 특성으로 인해 이미지 및 음성 인식, 자연어 처리 등과 같은 다양한 응용 분야에서 사용됩니다.

## Key Concepts 

* **인공신경망**: 딥러닝의 핵심 개념으로, 인간의 뇌를 모방한 구조로 데이터를 처리하고 학습합니다. 
* **심층 신경망**: 여러 개의 층으로 구성된 신경망으로, 데이터를 여러 단계로 처리하여 높은 수준의 추상화를 달성합니다. 
* **활성화 함수**: 신경망의 각 층에서 사용되는 함수로, 입력 데이터를 변환하여 출력합니다. 
* **최적화 알고리즘**: 신경망의 가중치를 업데이트하여 손실을 최소화하는 알고리즘으로, 학습 과정에서 사용됩니다.

## Applications 

* **이미지 인식**: 딥러닝은 이미지 인식 분야에서 뛰어난 성능을 발휘합니다. 예를 들어, 자율 주행 자동차의 카메라 인식, 얼굴 인식, 객체 탐지 등이 있습니다. 
* **자연어 처리**: 딥러닝은 자연어 처리 분야에서 사용됩니다. 예를 들어, 언어 번역, 감정 분석, 챗봇 등이 있습니다. 
* **음성 인식**: 딥러닝은 음성 인식 분야에서 사용됩니다. 예를 들어, 음성 비서, 음성 번역 등이 있습니다.

## Advantages 

* **높은 정확도**: 딥러닝은 복잡한 패턴을 학습할 수 있어 높은 정확도를 달성할 수 있습니다. 
* **대규모 데이터 처리**: 딥러닝은 대규모 데이터를 처리할 수 있어 빅데이터 분석에 적합합니다. 
* **다양한 응용 분야**: 딥러닝은 다양한 응용 분야에서 사용될 수 있어 그 활용 범위가 넓습니다.

## Challenges 

* **데이터 품질**: 딥러닝은 데이터 품질에 민감합니다. 따라서 데이터의 정확성과 완전성이 중요합니다. 
* **계산 자원**: 딥러닝은 대규모 계산

### FewShotPromptTemplate
* 예시를 제공 프롬프트

In [18]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
#result = chain.invoke({"input": "양자 얽힘이 무엇인가요?"})
print(result.content)

### 태양계의 행성
1. **수성**: 태양과 가장 가까운 행성으로, 매우 작고 온도가 극심하게 변합니다.
2. **금성**: 밝고 뜨거운 행성으로, 강한 온실 효과로 표면 온도가 매우 높습니다.
3. **지구**: 생명체가 살고 있는 행성으로, 물과 대기가 존재합니다.
4. **화성**: 붉은 행성으로, 로봇 탐사가 활발히 이루어지고 있습니다.
5. **목성**: 태양계에서 가장 큰 행성으로, 가스 거인입니다.
6. **토성**: 아름다운 고리를 가진 가스 거인입니다.
7. **천왕성**: 자전축이 기울어져 있는 얼음 거인입니다.
8. **해왕성**: 가장 먼 행성으로, 강한 바람과 추운 온도를 가지고 있습니다.


### PartialPromptTemplate
* 프롬프트의 입력 값에 함수 호출이나 외부 API를 호출한 동적인 값을 대입할 수 있음음

In [19]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month
    
    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}입니다.",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season("south")}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")  # '태풍 발생'은 여름과 관련됨
result = model.invoke(query)


# 결과 출력
print(f"🔹 프롬프트: {query}")
print(f"🔹 모델 응답: {result.content}")

🔹 프롬프트: 겨울에 일어나는 대표적인 지구과학 현상은 태풍 발생입니다.
🔹 모델 응답: 겨울에 태풍이 발생하는 것은 드문 일입니다. 태풍은 일반적으로 여름과 초가을에 발생합니다. 겨울에는 태풍이 발생하지 않는 것은 아니지만, 매우 드문 현상입니다. 겨울에 태풍이 발생하는 경우는 주로 중위도 지역에서 발생하는 저기압이나 폭풍우가 태풍으로 발달하는 경우입니다. 이러한 경우, 태풍의 세기는 일반적으로 약하고 지속시간도 짧습니다.

겨울에 태풍이 발생하는 대표적인 지역은 일본, 한국, 중국 등의 동아시아 지역입니다. 이 지역에서는 겨울에 저기압이 자주 발생하고, 이 저기압이 태풍으로 발달하는 경우가 있습니다.

그러나 겨울에 태풍이 발생하는 것은 여전히 드문 일이며, 대부분의 경우 태풍의 세기는 약하고 지속시간도 짧습니다. 따라서 겨울에 태풍이 발생하더라도 큰 피해를 입을 가능성은 낮습니다.


In [20]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# {info} 변수에 API에서 받은 환율 정보를 동적으로 반영

prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국경제 미치는 영향 및 향후에 환율의 예상값에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)
print(prompt)

input_variables=[] input_types={} partial_variables={'info': '1달러 = 1365.14원'} template='현재 {info} 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국경제 미치는 영향 및 향후에 환율의 예상값에 대한 분석을 제공해 주세요.'


In [22]:
# LLM 모델 설정 (GPT-4o-mini 사용)
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# 모델에 프롬프트 전달 및 응답 받기
response = model.invoke(prompt.format())

# 결과 출력
print("🔹 프롬프트:", prompt.format())
print("🔹 모델 응답:", response.content)

🔹 프롬프트: 현재 1달러 = 1365.14원 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국경제 미치는 영향 및 향후에 환율의 예상값에 대한 분석을 제공해 주세요.
🔹 모델 응답: ## 한국 경제에 미치는 영향

*   **수출 증가:** 약한 원화 가치는 한국의 수출을 촉진할 수 있습니다. 원화 약세는 한국 상품의 가격 경쟁력을 높여 해외 시장에서 판매량을 증가시킬 수 있습니다. 이는 자동차, 반도체, 철강 등의 산업에 긍정적인 영향을 미칠 수 있습니다.
*   **수입 비용 증가:** 원화 약세는 수입 물가를 상승시킬 수 있습니다. 원유, 원자재, 식품 등의 수입 비용이 증가하면 국내 물가 상승으로 이어질 수 있습니다. 이는 소비자들의 구매력을 감소시키고, 기업의 생산 비용을 증가시킬 수 있습니다.
*   **물가 상승:** 수입 물가의 상승은 국내 물가 상승으로 이어질 수 있습니다. 이는 소비자들의 생활비 부담을 증가시키고, 경제 성장률을 둔화시킬 수 있습니다.
*   **외국인 투자 감소:** 원화 약세는 외국인 투자자들에게 한국 시장에 대한 불안감을 조성할 수 있습니다. 이는 외국인 투자의 감소로 이어질 수 있으며, 이는 경제 성장률에 부정적인 영향을 미칠 수 있습니다.

## 향후 환율 예상

*   **글로벌 경제 상황:** 미국 연방준비제도(Fed)의 금리 인상, 중국 경제 성장 둔화, 유럽 경제 불확실성 등 글로벌 경제 상황은 원달러 환율에 큰 영향을 미칠 수 있습니다.
*   **한국 경제 상황:** 한국 경제 성장률, 물가 상승률, 무역 수지 등 한국 경제 상황도 원달러 환율에 영향을 미칠 수 있습니다.
*   **원화의 저평가 여부:** 원화가 저평가되어 있다면, 향후 원화 가치가 상승할 가능성이 있습니다. 하지만, 원화가 고평가되어 있다면, 향후 원화 가치가 하락할 가능성이 있습니다.

환율은 다양한 요인에 의해 변동하기 때문에 정확한 예측은 어렵습니다. 하지만, 최근 경제 상황과 전문가들의 분석을 바탕으로 향후 환율 전